### Quick Look

In [1]:
import os
baudrillard = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulacra-and-simulation.txt'
baudrillard_2 = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulations.txt'
def load_data(path: str) -> str:

    text_file = os.path.join(path)
    with open(text_file, "r") as f:
        data = f.read()

    return data

_data = load_data(baudrillard)
_data_2 = load_data(baudrillard_2)
main_data = _data + _data_2

In [2]:
import numpy as np

view_line_range = (100, 150)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in main_data.split()})))

lines = main_data.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(main_data.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 14440
Number of lines: 11204
Average number of words in each line: 7.909139593002499

The lines 100 to 150:
whose operation is nuclear and genetic, no longer at all specular or discursive. It is all of 
metaphysics that is lost. No more mirror of being and appearances, of the real and its 
concept. No more imaginary coextensivity: it is genetic miniaturization that is the 
dimension of simulation. The real is produced from miniaturized cells, matrices, and 
memory banks, models of control - and it can be reproduced an indefinite number of 
times from these. It no longer needs to be rational, because it no longer measures itself 
against either an ideal or negative instance. It is no longer anything but operational. In 
fact, it is no longer really the real, because no imaginary envelops it anymore. It is a 
hyperreal, produced from a radiating synthesis of combinatory models in a hyperspace 
without atmosphere. 

By crossing into a spac

### More Developed Dive

In [3]:
from utils import data

baudrillard = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulacra-and-simulation.txt'
baudrillard_2 = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulations.txt'

text = data.convertManyToOne([baudrillard, baudrillard_2])

tmp_path = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/tmp'
tmp_file = 'tmp_combined.txt'

if not os.path.exists(tmp_path):
    os.mkdir(tmp_path)

with open(f'{tmp_path}/{tmp_file}', 'w') as f:

    f.write(text)
    f.close()

data.preprocess_and_save(
    text_path=f'{tmp_path}/{tmp_file}',
    text_title='baudrillard_combined'
)
int_text, w2i, i2w, token_dict = data.load_preprocess('baudrillard_combined')

In [4]:
import torch

train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CPU')

CPU


In [5]:
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

def batch_data(
    words: list, 
    sequence_length: int, 
    batch_size: int) -> DataLoader:

    words = np.array(words)
    total_batch_size = batch_size * sequence_length
    n_batches = len(words) // batch_size

    words = words[:n_batches * total_batch_size]
    feature, target = [], []

    target_words_length = words[:-sequence_length]
    for idx in range(0, len(target_words_length)):

        feature.append(words[idx: idx + sequence_length])
        target.append(words[idx + sequence_length])

    batch_nums = len(words) // batch_size
    feature = feature[:batch_nums * batch_size]
    target = target[:batch_nums * batch_size]

    feature_tensors = torch.from_numpy(np.asarray(feature))
    target_tensors = torch.from_numpy(np.asarray(target))

    data_set = TensorDataset(feature_tensors, target_tensors)
    data_loader = DataLoader(data_set, batch_size=batch_size, shuffle=True)

    return data_loader

In [6]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[10, 11, 12, 13, 14],
        [ 1,  2,  3,  4,  5],
        [ 4,  5,  6,  7,  8],
        [40, 41, 42, 43, 44],
        [ 2,  3,  4,  5,  6],
        [33, 34, 35, 36, 37],
        [ 6,  7,  8,  9, 10],
        [31, 32, 33, 34, 35],
        [44, 45, 46, 47, 48],
        [32, 33, 34, 35, 36]])

torch.Size([10])
tensor([15,  6,  9, 45,  7, 38, 11, 36, 49, 37])


In [7]:
n_loader = batch_data(int_text, sequence_length=7, batch_size=32)

In [8]:
data_iter = iter(n_loader)
x,y = data_iter.next()

print(x)
print(y)

tensor([[  146,     1,  2926,    11,  2927,     0,   941],
        [10254,  2060,  3331,    18,  5361,    44,    31],
        [  154,    37,   548,   141,    12,    29,   587],
        [    1,     0,  8050,     4,     1,  8051,   645],
        [   14,     4,  2877,    14, 13406, 13407,   393],
        [ 2455, 14909, 14910,    27,     0,   710, 14911],
        [   24,   301,     5,  2252, 12043,  1008,   692],
        [    5,  9892,     0,   163,     2,     7,     1],
        [ 1501,    19,  9300,    23,   143,  9301,    19],
        [ 4557,     1,   618,     1,    33,    70,   719],
        [  744,    57,     1,     0,   239,  7156,  2843],
        [ 8487,     4,  8488,   250,  3162,    17,     9],
        [   77,   918,     0,   598,     7,  1259,    48],
        [    6,     0,   355,     4,    90,  2554,    38],
        [    1,     0,  2952,     1,     0,  1427,     1],
        [ 1351,     0,   164,    18,     0,  3016,  1068],
        [  779,    16,   151,     1,     0,  1865,    59